# Protein Design Agent with Strands

이 노트북에서 우리는 오픈 소스 Strands Agents 프레임워크를 사용하여 단백질 설계 에이전트를 만들어요.

#### Install Strands agents and required dependencies

In [ ]:
%pip install strands-agents strands-agents-tools --quiet

#### Ensure the latest version of boto3 is shown below
Ensure the boto3 version printed below is **1.37.1** or higher.

In [ ]:
%pip show boto3

#### Import required libraries

In [ ]:
import boto3
import json
import time
import uuid
import os
from typing import Dict, Any, Optional
from strands import Agent, tool
from strands.models import BedrockModel
from urllib.parse import urlparse

# Get AWS account information
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']
region = boto3.Session().region_name

## Prerequisites

Run through the notebook environment setup in [00-setup_environment.ipynb](00-setup_environment.ipynb).

Also ensure you have deployed the protein design CloudFormation stack from `stacks/protein_design_stack.yaml`

#### Setup AWS clients and configuration
Define the clients to AWS services that will be used by tools.

In [ ]:
# Initialize AWS clients
bedrock_client = boto3.client('bedrock-runtime', region_name=region)
omics_client = boto3.client('omics')
s3_client = boto3.client('s3')

# Configuration - Update these values based on your CloudFormation stack outputs
STACK_NAME = 'pppp'  # Your CloudFormation stack name
DEFAULT_WORKFLOW_ID = None  # Will be retrieved from stack outputs
DEFAULT_ROLE_ARN = None     # Will be retrieved from stack outputs
DEFAULT_S3_BUCKET = None    # Will be retrieved from stack outputs

# Set environment variable for tools to use
os.environ['STACK_NAME'] = STACK_NAME

print(f"Region: {region}")
print(f"Account ID: {account_id}")
print(f"Stack Name: {STACK_NAME}")

#### Get CloudFormation stack outputs

In [ ]:
# Get CloudFormation stack outputs
cf_client = boto3.client('cloudformation')

try:
    response = cf_client.describe_stacks(StackName=STACK_NAME)
    stack = response['Stacks'][0]

    # Get from outputs
    outputs = stack.get('Outputs', [])
    for output in outputs:
        key = output['OutputKey']
        value = output['OutputValue']
        if key == 'WorkflowId':
            DEFAULT_WORKFLOW_ID = value
        elif key == 'WorkflowExecutionRoleArn':
            DEFAULT_ROLE_ARN = value
            
    parameters = stack.get('Parameters', [])
    for param in parameters:
        if param['ParameterKey'] == 'S3BucketName':
            DEFAULT_S3_BUCKET = param['ParameterValue']
            break
    
    print(f"Workflow ID: {DEFAULT_WORKFLOW_ID}")
    print(f"Role ARN: {DEFAULT_ROLE_ARN}")
    print(f"S3 Bucket: {DEFAULT_S3_BUCKET}")
    
    # Configure the tools with the stack information
    from utils.protein_design_tools import set_stack_config
    set_stack_config(
        stack_name=STACK_NAME,
        workflow_id=DEFAULT_WORKFLOW_ID,
        role_arn=DEFAULT_ROLE_ARN,
        s3_bucket=DEFAULT_S3_BUCKET
    )
    
except Exception as e:
    print(f"Error getting stack outputs: {e}")
    print("Please update the configuration variables manually")

# Strands Agent Creation
In this section we create the agent using the Strands framework

#### Define agent configuration and instructions

In [ ]:
protein_agent_name = 'Protein-Design-Agent-Strands'
protein_agent_description = "Protein design and optimization agent using AWS HealthOmics workflows with Strands framework"
protein_agent_instruction = """
You are a protein design specialist AI that helps researchers optimize protein sequences using directed evolution algorithms.
You can trigger AWS HealthOmics workflows to perform protein sequence optimization and monitor their progress.

Your capabilities include:
1. Starting protein design optimization workflows with custom parameters
2. Monitoring the status of running workflows
3. Retrieving and analyzing results from completed optimizations

When working with protein sequences:
- Validate that sequences contain only valid amino acid letters
- Provide clear explanations of the optimization process
- Help users understand the results and their implications

Always be helpful and provide detailed information about the protein design process.
"""

#### Define tools for Strands agent
These tools will invoke AWS HealthOmics workflows for protein design operations

In [ ]:
# Import the updated tools that match the Bedrock agent action groups
from utils.protein_design_tools import trigger_aho_workflow, monitor_aho_workflow, test_configuration

# Set environment variables for the tools to use
#import os
#os.environ['DEFAULT_WORKFLOW_ID'] = DEFAULT_WORKFLOW_ID
#os.environ['DEFAULT_ROLE_ARN'] = DEFAULT_ROLE_ARN
#os.environ['DEFAULT_S3_BUCKET'] = DEFAULT_S3_BUCKET

print("Tools imported successfully:")
print(f"- trigger_aho_workflow: {trigger_aho_workflow.__doc__.split('Args:')[0].strip()}")
print(f"- monitor_aho_workflow: {monitor_aho_workflow.__doc__.split('Args:')[0].strip()}")

#### Create the Strands Agent

In [ ]:
# Create the Bedrock model
model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    client=bedrock_client
)

# Create the agent with tools that match the Bedrock agent action groups
protein_agent = Agent(
    system_prompt=protein_agent_instruction,
    model=model,
    tools=[trigger_aho_workflow, monitor_aho_workflow]
)

print(f"Created Protein Design Agent with Strands framework")

# Test the Agent
Let's test the protein design agent with some example queries

In [ ]:
#https://strandsagents.com/latest/documentation/docs/user-guide/observability-evaluation/logs/
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

#### Test 1: Start a protein optimization

In [ ]:
# Test starting a protein optimization
test_sequence = "EVQLVETGGGLVQPGGSLRLSCAASGFTLNSYGISWVRQAPGKGPEWVSVIYSDGRRTFYGDSVKGRFTISRDTSTNTVYLQMNSLRVEDTAVYYCAKGRAAGTFDSWGQGTLVTVSS"

query = f"Can you optimize this protein sequence: {test_sequence}"

print("Query:", query)

try:
    # Run the agent
    print("\nResponse:")
    protein_agent(query)
except Exception as e:
    print(f"Error during agent execution: {e}")
    import traceback
    traceback.print_exc()

#### Test 2: Monitor workflow status

In [ ]:
# Test monitoring a workflow (use a run ID from the previous test)
# Replace 'YOUR_RUN_ID' with an actual run ID from the previous test
test_run_id = "{test_run_id}"  # Update this with a real run ID

query = f"Check the status of workflow run {test_run_id}"

print("Query:", query)
print("\nResponse:")
response = protein_agent(query)

#### Test 3: Advanced optimization with custom parameters

In [ ]:
# Test with custom parameters
query = "Run a protein optimization for sequence ACDEFGHIKLMNPQRSTVWY with 20 parallel chains and 200 steps"

print("Query:", query)
print("\nResponse:")
response = protein_agent(query)

In [ ]:
# metrics: https://strandsagents.com/latest/documentation/docs/user-guide/observability-evaluation/metrics/

# Access metrics through the AgentResult
print(f"Total tokens: {response.metrics.accumulated_usage['totalTokens']}")
print(f"Execution time: {sum(response.metrics.cycle_durations):.2f} seconds")
print(f"Tools used: {list(response.metrics.tool_metrics.keys())}")


In [ ]:
# Install Bedrock AgentCore if not already installed
%pip install bedrock-agentcore bedrock-agentcore-starter-toolkit --quiet

In [ ]:
from bedrock_agentcore_starter_toolkit import deploy_agent

# Deploy the agent to Bedrock AgentCore
try:
    deployment_result = deploy_agent(
        agent=protein_agent,
        agent_name=protein_agent_name,
        description=protein_agent_description
    )
    
    print("Agent deployed successfully to Bedrock AgentCore!")
    print(f"Agent ID: {deployment_result.get('agent_id')}")
    print(f"Agent ARN: {deployment_result.get('agent_arn')}")
    
except Exception as e:
    print(f"Deployment failed: {e}")
    print("Make sure you have the necessary permissions and Bedrock AgentCore is available in your region")